In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
churn_data = pd.read_csv(r'C:\Users\Achintj\Python Code\Data Science\bank-customer-churn-modeling\Churn_Modelling.csv')

In [4]:
churn_data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [8]:
feature_data = churn_data.drop(['RowNumber','CustomerId','Surname','Exited'],axis = 1)
feature_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [10]:
target_data = churn_data.Exited
target_data.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [60]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

In [61]:
class CustLabel(BaseEstimator, TransformerMixin):
    
    def fit(self,X,Y=None):
        return self
    
    def transform(self,X,Y=None):
        db = {'Female':0,'Male':1}
        r = X.replace(db)
        return pd.DataFrame(r)

In [63]:
num_pipeline = make_pipeline(Normalizer())

In [64]:
label_pipeline = make_pipeline(CustLabel())

In [65]:
onehot_pipeline = make_pipeline(OneHotEncoder())

In [66]:
ct = ColumnTransformer(
    transformers = [
        ('num_data',num_pipeline,['CreditScore','Balance','EstimatedSalary']),
        ('label_data',label_pipeline,['Gender']),
        ('onehot_data', onehot_pipeline, ['Geography','NumOfProducts','HasCrCard','IsActiveMember'])
    ])

In [67]:
trainX,testX,trainY,testY = train_test_split(feature_data,target_data)

In [68]:
pipeline = make_pipeline(ct,RandomForestClassifier(n_estimators=100))

In [69]:
pipeline.fit(trainX,trainY)

Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num_data', Pipeline(memory=None,
     steps=[('normalizer', Normalizer(copy=True, norm='l2'))]), ['CreditScore', 'Balance', 'EstimatedSalary...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [71]:
pipeline.score(testX,testY)

0.8044

In [76]:
pipeline.steps

[('columntransformer',
  ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
           transformer_weights=None,
           transformers=[('num_data', Pipeline(memory=None,
       steps=[('normalizer', Normalizer(copy=True, norm='l2'))]), ['CreditScore', 'Balance', 'EstimatedSalary']), ('label_data', Pipeline(memory=None, steps=[('custlabel', CustLabel())]), ['Gender']), ('onehot_data', Pipeline(memory=None,
       steps=[('...     n_values=None, sparse=True))]), ['Geography', 'NumOfProducts', 'HasCrCard', 'IsActiveMember'])])),
 ('randomforestclassifier',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
          

In [73]:
from sklearn.model_selection import GridSearchCV

In [99]:
param_grid = {
    'randomforestclassifier__n_estimators': [10,20,100,1000],
    'randomforestclassifier__random_state': [1,10,50],
    'randomforestclassifier__max_depth': [1,10,50],
    'randomforestclassifier__n_jobs':[-1]
}

In [100]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, iid=False)
grid_search.fit(trainX,trainY)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num_data', Pipeline(memory=None,
     steps=[('normalizer', Normalizer(copy=True, norm='l2'))]), ['CreditScore', 'Balance', 'EstimatedSalary...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'randomforestclassifier__n_estimators': [10, 20, 100, 1000], 'randomforestclassifier__random_state': [1, 10, 50], 'randomforestclassifier__max_depth': [1, 10, 50], 'randomforestclassifier__n_jobs': [-1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [101]:
grid_search.best_params_

{'randomforestclassifier__max_depth': 10,
 'randomforestclassifier__n_estimators': 100,
 'randomforestclassifier__n_jobs': -1,
 'randomforestclassifier__random_state': 1}

In [102]:
grid_search.score(testX,testY)

0.8316

In [105]:
pred_y = grid_search.predict(testX)

In [106]:
from sklearn.metrics import accuracy_score

In [107]:
accuracy_score(testY,pred_y)

0.8316

In [111]:
testY[:10],pred_y[:10].reshape(-1,1)

(6585    0
 3149    0
 5982    0
 4089    0
 6961    0
 7998    1
 7710    0
 1118    0
 8618    0
 6536    0
 Name: Exited, dtype: int64, array([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0]], dtype=int64))